## 필요한 라이브러리 로드

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from types import SimpleNamespace
from models import DLinear, NLinear, LSTM

## 전체 데이터 로드

이 코드는 **모델 학습 전에 데이터의 스케일(척도)을 표준화하여 모델의 학습 안정성과 성능을 향상**시키기 위함. 특히, 시계열 데이터의 훈련 세트와 테스트 세트를 각각 스케일링하는 과정임.

  * **스케일러 객체 생성:** `scaler = MinMaxScaler()`를 사용하여 **`MinMaxScaler` 객체**를 생성함. `MinMaxScaler`는 데이터를 **0과 1 사이**의 값으로 정규화하는 방식임.
  * **훈련 데이터 스케일링 (Fit & Transform):** `train_data_scaled = scaler.fit_transform(train_data)`를 수행함.
      * **`fit()`:** 훈련 데이터(`train_data`)의 최소값(Min)과 최대값(Max)을 계산하여 스케일러가 이 정보를 학습하게 함.
      * **`transform()`:** 학습된 Min/Max 정보를 사용하여 훈련 데이터를 0과 1 사이로 변환함.
  * **테스트 데이터 스케일링 (Transform Only):** `test_data_scaled = scaler.transform(test_data)`를 수행함.
      * **`transform()`:** 훈련 데이터에서 학습한 Min/Max 값을 그대로 사용하여 테스트 데이터(`test_data`)를 변환함. 이는 데이터 누수(Data Leakage)를 방지하고 실제 환경과 동일하게 **훈련된 기준으로만** 새로운 데이터를 처리하도록 하기 위함.

In [14]:
raw_data = pd.read_csv('품질전처리후데이터.csv', index_col=0)
raw_data = raw_data.set_index('TAG_MIN')
raw_data.index = pd.to_datetime(raw_data.index)
raw_data = raw_data.resample('10S').mean()
raw_data = raw_data.dropna()

## 데이터 분할: 학습-검증-테스트

이 코드는 **특정 배정번호 데이터를 분리**한 후, 나머지 데이터를 **배정번호 기준**으로 **훈련(Train), 검증(Validation), 테스트(Test)** 데이터셋으로 **순차적으로 분할**함. 이는 시계열 데이터가 아닌, 배정번호(Batch ID)를 기준으로 데이터를 분리하는 독특한 방식임.

  * **특정 데이터 분리:**

      * `data = raw_data[raw_data['배정번호']!=148069]`를 통해 **마지막 배정번호**로 추정되는 `148069`번을 제외한 모든 데이터를 `data`에 할당함.
      * `data_for_sim = raw_data[raw_data['배정번호']==148069]`를 통해 **`148069`번 데이터**를 따로 `data_for_sim`에 저장함. 이는 아마도 모델 학습/평가와 별개로 **시뮬레이션(Simulation)** 등을 위해 해당 배치를 별도로 활용하기 위함임.

  * **훈련, 검증, 테스트 분할 (배정번호 기준):**

      * **분할 기준점 설정:** 데이터의 길이 비율(70%, 80%)을 사용하여 해당 위치의 **배정번호 값**을 추출함.
          * `data.iloc[int(len(data)*0.7), 0]` : 전체 `data`의 **70% 지점**에 있는 행의 **첫 번째 컬럼 (배정번호)** 값을 분할 기준으로 사용함.
          * `data.iloc[int(len(data)*0.8), 0]` : 전체 `data`의 **80% 지점**에 있는 행의 **첫 번째 컬럼 (배정번호)** 값을 분할 기준으로 사용함.
      * **`train_data` (훈련 데이터):** `data`에서 **70% 지점의 배정번호보다 작은** 모든 데이터를 훈련 세트로 사용함.
      * **`vali_data` (검증 데이터):** `data`에서 **70% 지점의 배정번호**와 **80% 지점의 배정번호 사이**에 있는 데이터를 검증 세트로 사용함.
      * **`test_data` (테스트 데이터):** `data`에서 **80% 지점의 배정번호보다 크거나 같은** 모든 데이터를 테스트 세트로 사용함.

  * **목적:** 데이터의 **시간 순서** 대신 **생산 배치(배정번호)** 순서를 기준으로 데이터를 분할하여, 모델이 과거 배치들을 학습하고 새로운 배치들을 검증 및 테스트하도록 함. 이는 시계열 데이터가 배치 단위로 독립적인 특성을 가질 때 유용함.

In [15]:
# 마지막 배정번호 제외
data = raw_data[raw_data['배정번호']!=148069]
data_for_sim = raw_data[raw_data['배정번호']==148069]

train_data = data[data['배정번호']<data.iloc[int(len(data)*0.7),0]]
vali_data = data[(data['배정번호']>=data.iloc[int(len(data)*0.7),0]) & (data['배정번호']<data.iloc[int(len(data)*0.8),0])]
test_data = data[data['배정번호']>=data.iloc[int(len(data)*0.8),0]]

len(train_data), len(vali_data), len(test_data)

(204088, 31782, 61333)

## 정규화: StandardScaling 및 Scaler 저장

이 코드는 **Scikit-learn의 `StandardScaler`를 사용하여 훈련, 검증, 테스트 데이터의 특정 컬럼들을 표준화**하여 모델 학습에 적합하게 스케일링하는 과정임.

  * **StandardScaler 사용:** `StandardScaler()`를 사용하여 스케일러 객체를 생성함. 이 스케일러는 데이터를 **평균이 0, 표준편차가 1**이 되도록 표준화함.
  * **특정 컬럼만 스케일링:** 모든 데이터에 대해 `.iloc[:, 1:]` 인덱싱을 사용함. 이는 첫 번째 컬럼(인덱스 0)을 제외한 **나머지 모든 컬럼**을 스케일링 대상으로 지정함을 의미함. (일반적으로 첫 번째 컬럼은 스케일링이 필요 없는 `배정번호`와 같은 식별자 데이터일 수 있음).
  * **훈련 데이터 학습 및 변환:** `train_data_scaled = scaler.fit_transform(train_data.iloc[:, 1:])`를 통해 훈련 데이터의 **평균과 표준편차를 학습**(`fit`)하고 이를 기준으로 데이터를 표준화(`transform`)함.
  * **검증/테스트 데이터 변환:** `scaler.transform()`만 사용하여 검증(`vali_data`) 및 테스트(`test_data`) 데이터를 변환함. 이는 훈련 데이터에서 학습된 통계 정보(평균, 표준편차)를 사용하여 새로운 데이터들을 처리함으로써, **데이터 누수(Data Leakage)를 방지**함.
  * **원래 데이터에 적용:** 마지막 세 줄을 통해 스케일링된 값(`*_data_scaled`)을 원래의 데이터프레임(`*_data`)의 해당 컬럼(`iloc[:, 1:]`)에 **대입하여 업데이트**함.
  * **Scaler 저장:** `pickle`을 사용하여 학습된 scaler 객체를 `scaler.pkl` 파일로 저장함. 이는 **dashboard에서 예측 결과를 역정규화**하기 위해 필요함.

In [ ]:
# Standard scaling
from sklearn.preprocessing import StandardScaler
import pickle

scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data.iloc[:,1:])
vali_data_scaled = scaler.transform(vali_data.iloc[:,1:])
test_data_scaled = scaler.transform(test_data.iloc[:,1:])

train_data.iloc[:,1:] = train_data_scaled
vali_data.iloc[:,1:] = vali_data_scaled
test_data.iloc[:,1:] = test_data_scaled

# Scaler 저장 (dashboard에서 역정규화에 사용)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print(f"StandardScaler 저장 완료: scaler.pkl")
print(f"  - 평균(mean): {scaler.mean_[:5]} (처음 5개)")
print(f"  - 표준편차(scale): {scaler.scale_[:5]} (처음 5개)")
print(f"  - 총 특성 수: {len(scaler.mean_)}")

## 시계열 데이터셋 정의

이 코드는 **시계열 예측 모델 학습을 위해 Pandas DataFrame 형태의 시계열 데이터를 PyTorch의 `Dataset` 객체로 변환**함. 이를 통해 데이터를 효율적으로 관리하고, **슬라이딩 윈도우(Sliding Window)** 방식을 적용하여 모델 입력에 필요한 시퀀스 형태로 데이터를 공급할 수 있게 됨.

  * **PyTorch `Dataset` 상속:** `torch.utils.data.Dataset` 클래스를 상속받아 시계열 데이터 전용 데이터 로딩 클래스를 정의함. 이는 PyTorch의 `DataLoader`와 함께 사용되어 데이터 미니배치(mini-batch)를 효율적으로 생성할 수 있게 함.
  * **시계열 시퀀스 정의 (`__init__`):**
      * `input_len`과 `output_len`을 받아 과거 `input_len` 길이의 데이터를 입력(`X`)으로, 미래 `output_len` 길이의 데이터를 예측 대상(`y`)으로 정의함.
      * **`task`** 인수를 통해 **다변량(multivariate)** 또는 **단변량(univariate)** 예측 문제를 선택함.
          * `multivariate`: 예측 대상(`target_name`)을 제외한 모든 컬럼을 피처(`features`)로 사용함.
          * `univariate`: 예측 대상 컬럼만 피처로 사용함.
  * **데이터 길이 계산 (`__len__`):**
      * `self.max_index`를 계산하여 유효한 슬라이딩 윈도우의 최대 개수를 정의함. 이 길이를 반환하여 `DataLoader`가 반복 횟수를 알 수 있게 함.
  * **슬라이딩 윈도우 구현 (`__getitem__`):**
      * `idx` (인덱스)가 주어지면, 해당 인덱스부터 `input_len` 길이만큼의 피처 데이터(`X`)를 잘라냄.
      * `input_len` 다음 위치부터 `output_len` 길이만큼의 타겟 데이터(`y`)를 잘라냄.
      * 잘라낸 NumPy 배열 데이터를 PyTorch **`torch.tensor`** 객체로 변환하여 반환함.

In [17]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, input_len, output_len, target_name, task="multivariate"):
        """
        시계열 데이터프레임을 torch Dataset으로 변환
        
        Args:
            df (pd.DataFrame): 시계열 데이터 (정렬되어 있어야 함)
            input_len (int): 입력 시퀀스 길이
            output_len (int): 출력(예측) 시퀀스 길이
            target_name (str): 예측 대상 컬럼명
            task (str): 'multivariate' 또는 'univariate'
                        - 'multivariate': 전체 feature 사용
                        - 'univariate': target feature만 입력으로 사용
        """
        assert isinstance(df, pd.DataFrame), "df는 pandas DataFrame이어야 합니다."
        assert target_name in df.columns, f"{target_name} 컬럼이 DataFrame에 없습니다."
        assert task in ["multivariate", "univariate"], "task는 'multivariate' 또는 'univariate' 중 하나여야 합니다."

        self.input_len = input_len
        self.output_len = output_len
        self.target_name = target_name
        self.task = task

        # feature 선택 로직
        if task == "multivariate":
            self.feature_cols = [c for c in df.columns if c != target_name]
        else:  # univariate
            self.feature_cols = [target_name]

        self.features = df[self.feature_cols].values.astype(np.float32)
        self.target = df[target_name].values.astype(np.float32)

        self.max_index = len(df) - input_len - output_len + 1

    def __len__(self):
        return max(0, self.max_index)

    def __getitem__(self, idx):
        """
        슬라이딩 윈도우 기반으로 (X, y) 반환
        X: (input_len, feature_dim)
        y: (output_len,)
        """
        X = self.features[idx : idx + self.input_len]
        y = self.target[idx + self.input_len : idx + self.input_len + self.output_len]

        return torch.tensor(X), torch.tensor(y)

## 변수별 데이터로더 생성

이 코드는 **각 예측 대상(Target) 변수별로 훈련, 검증, 테스트 데이터로더(DataLoader)를 생성하고 저장**함. 이때, **데이터의 불연속성을 처리하기 위해 각 배정번호(Batch ID) 단위로 데이터셋을 생성한 후 이를 병합**하는 방식을 사용함.

  * **예측 목표 및 시퀀스 길이 설정:**
      * `features` 리스트에는 모델이 예측할 **세 가지 목표 변수**를 정의함.
      * `input_len = 30`은 모델 입력 시퀀스 길이를 **30**으로, `output_len = 60`은 예측 시퀀스 길이를 **60**으로 설정함.
      * `task = 'univariate'`로 설정하여 **단변량 예측**을 수행하도록 지정함. (입력 피처로 타겟 변수 자체만 사용함)
  * **배정번호 단위 데이터셋 생성 및 병합:**
      * **`for target in features:`** 루프를 통해 각 타겟 변수(`'건조로 온도 2 Zone'`, `'소입2존 OP'`, `'소입로 온도 2 Zone'`)별로 다음 과정을 반복함.
      * **배정번호별 분리:** 훈련(`train_data`), 검증(`vali_data`), 테스트(`test_data`) 각 데이터셋을 **`배정번호`의 고유 값**으로 순회함.
      * **`TimeSeriesDataset` 생성:** 분리된 배정번호 단위의 DataFrame(`df`)을 **이전에 정의된 `TimeSeriesDataset` 클래스**를 이용해 데이터셋으로 변환함. 이는 **배정번호가 다르면 시간이 불연속적**이므로, 윈도우 슬라이딩이 배정번호 경계를 넘지 않도록 하기 위함임.
      * **데이터셋 병합:** 각 배정번호별로 생성된 데이터셋 리스트(`trainset_list`, `valiset_list`, `testset_list`)를 `torch.utils.data.ConcatDataset`을 사용하여 **하나의 큰 데이터셋**(`trainset`, `valiset`, `testset`)으로 합침.
  * **DataLoader 생성:**
      * 병합된 데이터셋을 PyTorch의 `DataLoader`로 변환하여 모델 학습 시 미니배치(mini-batch)를 제공할 준비를 함.
      * `batch_size=4096`으로 설정하여 배치 크기를 정의함.
      * 훈련 데이터로더(`trainloader`)에만 `shuffle=True`를 적용하여 학습 시 데이터 순서를 섞음. (검증 및 테스트 데이터로더는 `shuffle=False`로 순서를 유지함.)
  * **결과 저장:** 생성된 **훈련, 검증, 테스트 데이터로더** 튜플을 딕셔너리(`feature_dataloaders`)에 저장함. 이때 **타겟 변수명**을 딕셔너리의 키(Key)로 사용함.

In [18]:
features = ['건조로 온도 2 Zone', '소입로 온도 4 Zone', '소입로 온도 1 Zone']
input_len = 30
output_len = 60
task = 'univariate'
feature_dataloaders = {}
for target in features:
    trainset_list = []
    for id in train_data['배정번호'].unique():
        df = train_data[train_data['배정번호']==id]
        dataset = TimeSeriesDataset(df, input_len=input_len, output_len=output_len, target_name=target, task=task)
        trainset_list.append(dataset)
    trainset = torch.utils.data.ConcatDataset(trainset_list)

    valiset_list = []
    for id in vali_data['배정번호'].unique():
        df = vali_data[vali_data['배정번호']==id]
        dataset = TimeSeriesDataset(df, input_len=input_len, output_len=output_len, target_name=target, task=task)
        valiset_list.append(dataset)
    valiset = torch.utils.data.ConcatDataset(valiset_list)

    testset_list = []
    for id in test_data['배정번호'].unique():
        df = test_data[test_data['배정번호']==id]
        dataset = TimeSeriesDataset(df, input_len=input_len, output_len=output_len, target_name=target, task=task)
        testset_list.append(dataset)
    testset = torch.utils.data.ConcatDataset(testset_list)

    trainloader = DataLoader(trainset, batch_size=4096, shuffle=True)
    valiloader = DataLoader(valiset, batch_size=4096, shuffle=False)
    testloader = DataLoader(testset, batch_size=4096, shuffle=False)
    
    feature_dataloaders[target] = (trainloader, valiloader, testloader)

## 학습 및 테스트 함수 정의

이 코드는 **시계열 예측 모델의 학습, 검증, 최적 모델 저장, 그리고 성능 평가 및 결과 기록**을 위한 두 가지 핵심 함수(`train_model`, `test_model`)를 정의함.

### **1. `train_model`의 목적 및 상세 설명**

  * **목적:** 훈련 데이터셋을 사용하여 모델을 학습시키고, 검증 데이터셋에서 손실(Loss)을 모니터링하여 **최적의 성능을 보인 모델 가중치**를 저장하며, 최종적으로 해당 모델을 **ONNX 형식**으로 변환하여 저장함.
  * **주요 기능:**
      * **학습 루프:** `nn.MSELoss` 또는 사용자 정의 손실 함수를 기준으로 각 Epoch마다 훈련 및 검증을 수행하며 손실을 출력함.
      * **최적 모델 저장:** 검증 손실(`val_loss`)이 **가장 낮을 때**의 모델 가중치(`state_dict`)를 `best_state`에 저장함.
      * **ONNX 변환 및 저장:** 학습이 완료된 후, 가장 좋은 가중치를 로드하고 **`torch.onnx.export`** 함수를 사용하여 모델을 **`checkpoints/{file_name}`** 경로에 ONNX 형식으로 저장함. 이때 배치 크기를 가변적(`dynamic_axes`)으로 설정하여 범용성을 확보함.

### **2. `test_model`의 목적 및 상세 설명**

  * **목적:** 학습 완료된 모델을 테스트 데이터셋에 적용하여 **최종 예측 성능을 평가**하고, 그 결과를 파일에 기록함.
  * **주요 기능:**
      * **성능 지표 계산:** 테스트 데이터셋의 예측 결과와 실제 값(`preds_all`, `target_all`)을 사용하여 **MAE (Mean Absolute Error), RMSE (Root Mean Square Error), SMAPE (Symmetric Mean Absolute Percentage Error)** 세 가지 지표를 계산함.
      * **결과 기록 (`result.txt`):** 계산된 성능 지표를 `result.txt` 파일에 **추가(append) 모드**로 기록함.
      * **결과 기록 (`result.csv`):** 모델명과 성능 지표를 포함하는 행을 생성하여 **`result.csv`** 파일에 추가 기록함. (파일이 없으면 새로 생성하고, 있으면 헤더 없이 데이터만 추가함.)

In [19]:
def train_model(model, trainloader, valiloader, num_epochs=20, lr=1e-3, criterion=None, file_name='model.onnx'):
    """
    학습 및 검증 루프
    model: nn.Module
    trainloader, valiloader: DataLoader
    num_epochs: 학습 epoch 수
    lr: learning rate
    criterion: 손실 함수 (기본 MSE)
    """
    model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = criterion or nn.MSELoss()

    best_val_loss = float("inf")
    best_state = None

    for epoch in range(1, num_epochs + 1):
        # ---------------- TRAIN ----------------
        model.train()
        train_loss = 0.0
        for X, y in trainloader:
            X, y = X.cuda(), y.cuda()
            optimizer.zero_grad()

            preds = model(X)[:,:,0]  # [batch, pred_len, n_features]
            loss = criterion(preds, y)

            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.size(0)

        train_loss /= len(trainloader.dataset)

        # ---------------- VALIDATION ----------------
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X, y in valiloader:
                X, y = X.cuda(), y.cuda()
                preds = model(X)[:,:,0]
                loss = criterion(preds, y)
                val_loss += loss.item() * X.size(0)
        val_loss /= len(valiloader.dataset)

        print(f"[Epoch {epoch:02d}] Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

        # best model 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict()

    # 학습 종료 후 best model 로드
    if best_state:
        model.load_state_dict(best_state)
        print(f"Best model restored (val_loss={best_val_loss:.6f})")
        
    # 모델 저장
    model.eval() # 추론 모드로 설정
    model.cpu()
    torch.onnx.export(
    model,                      # 모델
    torch.randn(1, 30, 1) ,                # 모델에 한 번 통과시킬 더미 입력 (튜플 또는 텐서)
    f'checkpoints/{file_name}',             # 저장 경로 및 파일 이름
    export_params=True,         # 학습된 파라미터도 함께 저장
    opset_version=14,           # ONNX Operationset Version (일반적으로 최신 버전을 사용, 14 또는 17 권장)
    do_constant_folding=True,   # 상수 폴딩 최적화 수행
    input_names=['input'],      # 입력 노드 이름
    output_names=['output'],    # 출력 노드 이름
    dynamic_axes={
        'input': {0: 'batch_size'},   # 입력 텐서의 0번째 차원(배치)은 가변적
        'output': {0: 'batch_size'}  # 출력 텐서의 0번째 차원(배치)은 가변적
    }
)

    return model


def test_model(model, testloader, criterion=None, file_name='model.onnx'):
    """
    테스트 루프
    """
    model.eval()
    model.cuda()
    criterion = criterion or nn.MSELoss()
    test_loss = 0.0

    preds_list, target_list = [], []
    with torch.no_grad():
        for X, y in testloader:
            X, y = X.cuda(), y.cuda()
            preds = model(X)[:,:,0]
            loss = criterion(preds, y)
            test_loss += loss.item() * X.size(0)

            preds_list.append(preds.cpu())
            target_list.append(y.cpu())

    preds_all = torch.cat(preds_list)
    target_all = torch.cat(target_list)
    
    # 결과 출력: MAE, RMSE, SMAPE
    mae = torch.mean(torch.abs(preds_all - target_all)).item()
    rmse = torch.sqrt(torch.mean((preds_all - target_all) ** 2)).item()
    smape = torch.mean(2 * torch.abs(preds_all - target_all) / (torch.abs(preds_all) + torch.abs(target_all) + 1e-8)).item() * 100
    print(f"[{file_name} Test Results] MAE: {mae:.6f}, RMSE: {rmse:.6f}, SMAPE: {smape:.6f}%")
    
    # result.txt에 결과 추가 기록
    with open(f'result.txt', 'a') as f:
        f.write(f"[{file_name} Test Results]\n")
        f.write(f"MAE: {mae:.6f}\n")
        f.write(f"RMSE: {rmse:.6f}\n")
        f.write(f"SMAPE: {smape:.6f}%\n")
        f.write("\n")
    
    # result.csv에 결과 추가 기록
    row = [file_name[:-5], mae, rmse, smape]
    df = pd.DataFrame([row], columns=['Model', 'MAE', 'RMSE', 'SMAPE'])
    if not os.path.isfile(f'result.csv'):
        df.to_csv(f'result.csv', index=False)
    else:
        df.to_csv(f'result.csv', mode='a', header=False, index=False)

    return preds_all, target_all, (mae, rmse, smape)

## 모델 일괄 학습 및 테스트

이 코드는 **정의된 여러 모델(`DLinear`, `NLinear`, `LSTM`)과 하이퍼파라미터 조합**을 사용하여 **모든 타겟 변수**에 대해 **모델을 일괄적으로 학습시키고 테스트를 수행**하여 최적의 조합을 탐색함. 이는 **모델 비교 실험을 자동화**하는 핵심 루프임.

  * **실험 설정(`configs`):** `SimpleNamespace`를 사용하여 모델에 필요한 설정값(Configuration)을 정의함.
      * `seq_len=30`: 입력 시퀀스 길이 30.
      * `pred_len=60`: 예측 시퀀스 길이 60.
      * `enc_in=1`: 입력 피처 수 1 (이전 단계에서 `task='univariate'`로 설정했으므로 단변량 입력임).
  * **다중 루프를 통한 실험 조합 생성:** **4중 루프**를 사용하여 가능한 모든 실험 조합을 생성하고 반복함.
    1.  **`target` (예측 대상):** 각 타겟 변수별로 실험을 수행함. (`feature_dataloaders`에서 데이터로더를 가져옴)
    2.  **`model_name` (모델 구조):** `DLinear`, `NLinear`, `LSTM` 세 가지 모델을 순회함.
    3.  **`learning_rate` (학습률):** `1e-3` (0.001)과 `1e-2` (0.01) 두 가지 학습률을 사용함.
    4.  **`criterion_name` (손실 함수):** `L1Loss` (MAE)와 `MSELoss` (L2) 두 가지 손실 함수를 사용함.
  * **모델 초기화 및 파일명 설정:**
      * 각 루프마다 `model_dict[model_name].Model(configs)`를 호출하여 **새로운 모델**을 생성하고 GPU(`cuda()`)로 보냄.
      * 손실 함수(`criterion`)를 설정함.
      * `file_name`은 모델명, 타겟, 학습률, 손실 함수 이름이 포함된 고유한 이름으로 설정됨 (`DLinear_소입로 온도 2 Zone_0.001_L1Loss.onnx`와 같은 형식).
  * **학습 및 테스트 실행:**
      * `train_model` 함수를 호출하여 모델을 학습하고 검증 손실이 가장 낮은 모델을 ONNX 파일로 저장함.
      * `test_model` 함수를 호출하여 학습된 모델의 최종 성능을 테스트하고, 그 결과를 `result.txt`와 `result.csv` 파일에 **누적 기록**함.

In [20]:
model_dict = {
    'DLinear': DLinear,
    'NLinear': NLinear,
    'LSTM': LSTM,
}

configs = SimpleNamespace(
    seq_len=30,
    pred_len=60,
    moving_avg=25,
    individual=False,
    enc_in=1, # feature 수: 1 or 18
    dropout=0.1,
)

for target in features:
    trainloader, valiloader, testloader = feature_dataloaders[target]
    for model_name in model_dict.keys():
        for learning_rate in [1e-3, 1e-2]:
            for criterion_name in ['L1Loss', 'MSELoss']:
                model = model_dict[model_name].Model(configs).float().cuda()
                if criterion_name == 'MSELoss':
                    criterion = nn.MSELoss()
                else:
                    criterion = nn.L1Loss()
                file_name = f'{model_name}_{target}_{learning_rate}_{criterion_name}.onnx'

                # 학습 및 검증
                trained_model = train_model(model, trainloader, valiloader, num_epochs=10, lr=learning_rate, criterion=criterion, file_name=file_name)

                # 테스트
                preds, targets, test_loss = test_model(trained_model, testloader, criterion, file_name=file_name)

[Epoch 01] Train Loss: 0.420292 | Val Loss: 0.403872
[Epoch 02] Train Loss: 0.362063 | Val Loss: 0.386063
[Epoch 03] Train Loss: 0.358672 | Val Loss: 0.385080
[Epoch 04] Train Loss: 0.357572 | Val Loss: 0.382960
[Epoch 05] Train Loss: 0.357147 | Val Loss: 0.388940
[Epoch 06] Train Loss: 0.356462 | Val Loss: 0.383303
[Epoch 07] Train Loss: 0.355048 | Val Loss: 0.382110
[Epoch 08] Train Loss: 0.354508 | Val Loss: 0.381799
[Epoch 09] Train Loss: 0.355006 | Val Loss: 0.381396
[Epoch 10] Train Loss: 0.354208 | Val Loss: 0.380383
Best model restored (val_loss=0.380383)
[DLinear_건조로 온도 2 Zone_0.001_L1Loss.onnx Test Results] MAE: 0.357898, RMSE: 0.608660, SMAPE: 98.455328%
[Epoch 01] Train Loss: 0.505138 | Val Loss: 0.516851
[Epoch 02] Train Loss: 0.397197 | Val Loss: 0.459976
[Epoch 03] Train Loss: 0.376897 | Val Loss: 0.451258
[Epoch 04] Train Loss: 0.374099 | Val Loss: 0.441954
[Epoch 05] Train Loss: 0.371566 | Val Loss: 0.439976
[Epoch 06] Train Loss: 0.370147 | Val Loss: 0.444145
[Epoch 0

/home/proj03/.venv/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


[LSTM_건조로 온도 2 Zone_0.001_L1Loss.onnx Test Results] MAE: 0.347368, RMSE: 0.583053, SMAPE: 99.265420%
[Epoch 01] Train Loss: 0.742234 | Val Loss: 0.773990
[Epoch 02] Train Loss: 0.499409 | Val Loss: 0.518734
[Epoch 03] Train Loss: 0.391074 | Val Loss: 0.442906
[Epoch 04] Train Loss: 0.362365 | Val Loss: 0.412850
[Epoch 05] Train Loss: 0.350920 | Val Loss: 0.393919
[Epoch 06] Train Loss: 0.344095 | Val Loss: 0.389866
[Epoch 07] Train Loss: 0.340242 | Val Loss: 0.387417
[Epoch 08] Train Loss: 0.336201 | Val Loss: 0.377671
[Epoch 09] Train Loss: 0.333181 | Val Loss: 0.375636
[Epoch 10] Train Loss: 0.329712 | Val Loss: 0.374222
Best model restored (val_loss=0.374222)
[LSTM_건조로 온도 2 Zone_0.001_MSELoss.onnx Test Results] MAE: 0.348209, RMSE: 0.571146, SMAPE: 96.801245%
[Epoch 01] Train Loss: 0.436201 | Val Loss: 0.393733
[Epoch 02] Train Loss: 0.355289 | Val Loss: 0.371477
[Epoch 03] Train Loss: 0.345605 | Val Loss: 0.354748
[Epoch 04] Train Loss: 0.341173 | Val Loss: 0.360853
[Epoch 05] Trai

## 최고 성능 모델 확인

이 코드는 **실험 결과 파일(`result.csv`)을 읽어와서 모든 모델의 성능을 비교**하고, **각 타겟 변수별로 가장 좋은 성능 지표(MAE 기준)를 달성한 최적 모델 조합을 식별하고 정리**함.

  * **결과 데이터 로드:** `pd.read_csv('result.csv', index_col='Model')`을 사용하여 이전에 `test_model` 함수에서 누적 기록한 **모든 실험 결과**를 DataFrame으로 불러옴.
  * **인덱스 분해:** 이전 단계와 동일하게, 복합적인 인덱스(예: `LSTM_소입로 온도 2 Zone_0.001_L1Loss`)를 **`model`, `target`, `learning_rate`, `loss_type`** 네 가지 개별 컬럼으로 정확하게 분리함. 이는 각 하이퍼파라미터 조합별로 비교하기 위함임.
  * **최적 모델 선택 (MAE 기준):** `result_df.groupby('target')['MAE'].idxmin()`을 사용하여 각 타겟 변수 그룹 내에서 **가장 낮은 MAE 값**을 가진 행을 찾아냄. 낮은 MAE는 모델의 오차가 적다는 것을 의미하므로, **가장 좋은 성능**을 나타내는 기준으로 사용됨.
  * **최종 결과 정리 및 출력:**
      * 선택된 최적 모델들의 정보(`model`, `target`, `learning_rate`, `loss_type`, `MAE`, `RMSE`, `SMAPE`)만 `final_best_df`로 정리함.
      * 이 DataFrame을 Markdown 형식의 표로 출력하여 사용자가 **타겟별 최고 성능 모델**을 한눈에 확인할 수 있게 함.
  * **최적 ONNX 파일명 저장:**
      * `for target in features:` 루프를 통해 각 타겟별 최적 모델 행을 가져옴.
      * **모델, 타겟, 학습률, 손실 타입**을 언더바(`_`)로 다시 연결하고 `.onnx`를 붙여 **최고 성능 모델의 ONNX 파일명**을 생성함.
      * 이 파일명을 **`best_models` 딕셔너리**에 저장하여, 이후 시뮬레이션이나 실제 배포 시 어떤 파일(`*.onnx`)을 사용해야 하는지 명확히 함.

In [ ]:
result_df = pd.read_csv('result.csv', index_col='Model')

# 2. 인덱스(Model) 분할 및 새 컬럼 생성
model_parts = result_df.index.str.split('_', expand=True)

result_df['model'] = model_parts.get_level_values(0)

split_df = result_df.index.to_series().apply(lambda x: x.rsplit('_', 3))
result_df['model'] = split_df.apply(lambda x: x[0])
result_df['target'] = split_df.apply(lambda x: x[1].replace(' ', '_')) # Target에 띄어쓰기가 있다면 임시로 _로 변경
result_df['learning_rate'] = split_df.apply(lambda x: x[2])
result_df['loss_type'] = split_df.apply(lambda x: x[3])

# target 컬럼의 띄어쓰기를 원래대로 복구
result_df['target'] = result_df['target'].str.replace('_', ' ')

# 3. 각 Target 별로 MAE가 가장 낮은 행 선택
best_results_per_target = result_df.loc[result_df.groupby('target')['MAE'].idxmin()]

# 4. 최종 출력 컬럼 정리
output_columns = ['model', 'target', 'learning_rate', 'loss_type', 'MAE', 'RMSE', 'SMAPE']
final_best_df = best_results_per_target[output_columns].reset_index(drop=True)

best_models = {}
for target in features:
    model_row = final_best_df[final_best_df['target']==target].iloc[:,0:4]
    concatenated_name = '_'.join(model_row.values[0])
    best_model = concatenated_name + '.onnx'
    best_models[target] = best_model

print("타겟별 최적 조합 결과 (MAE 기준):\n")
print(final_best_df.to_markdown(index=False))

타겟별 최적 조합 결과 (MAE 기준):

| model   | target             |   learning_rate | loss_type   |      MAE |     RMSE |    SMAPE |
|:--------|:-------------------|----------------:|:------------|---------:|---------:|---------:|
| LSTM    | 건조로 온도 2 Zone |            0.01 | L1Loss      | 0.334453 | 0.562763 |  98.1504 |
| LSTM    | 소입로 온도 1 Zone |            0.01 | L1Loss      | 0.324851 | 0.669394 |  74.9307 |
| LSTM    | 소입로 온도 4 Zone |            0.01 | L1Loss      | 0.311645 | 0.455928 | 115.97   |
